In [1]:
import tensorflow as tf
import numpy as np
import random
from tqdm import tqdm

In [2]:
def unpickle( file ):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [3]:
def conv( x, filter_size=3, stride=2, num_filters=64, is_output=False, name="conv" ):
    shape = x.get_shape().as_list()
    with tf.name_scope(name) as scope:
        W = tf.get_variable("conv_{}_W".format(name), [filter_size, filter_size, x.shape[-1], num_filters], initializer = tf.contrib.layers.variance_scaling_initializer())
        B = tf.get_variable("conv_{}_B".format(name), [num_filters], initializer=tf.ones_initializer())
        conv = tf.nn.conv2d(x, W, [1, stride, stride, 1] , "SAME")
        res = tf.nn.bias_add(conv, B)
        #print "conv ", conv.shape, "res ", res.shape
        if not is_output:
            res = tf.nn.relu(res)
        return res
    
    '''
    x is an input tensor
    Declare a name scope using the "name" parameter
    Within that scope:
      Create a W filter variable with the proper size
      Create a B bias variable with the proper size
      Convolve x with W by calling the tf.nn.conv2d function
      Add the bias
      If is_output is False,
        Call the tf.nn.relu function
      Return the final op
    ''' 
    pass

In [4]:
data = unpickle( './cifar-10-batches-py/data_batch_1' )
c = .00001 #the learning rate
features = data['data']
features = (features-np.mean(features,axis=0))/np.std(features,axis=0)
labels = data['labels']
labels = np.atleast_2d( labels ).T

resized_features = []
for x in xrange(features.shape[0]):
    resized_features.append(np.reshape(features[x], [1,32,32,3]))

resized_features = np.array(resized_features)

#model_selection.train_test_split
split = int(features.shape[0]*.8)
train_features = resized_features[:split]
train_labels = labels[:split]
test_features = resized_features[split:]
test_labels = labels[split:]

IOError: [Errno 2] No such file or directory: './cifar-10-batches-py/data_batch_1'

In [ ]:

def fc( x, out_size=50, name="fc" , is_output=False, bias_val=0.0):
    
    shape = x.get_shape().as_list()
    
    with tf.name_scope(name) as scope:
        #W = tf.get_variable("fc_{}_W".format(name), [in_size, out_size],initializer = tf.contrib.layers.variance_scaling_initializer())
        W = tf.get_variable("fc_{}_W".format(name), [shape[1], out_size], tf.float32, initializer = tf.contrib.layers.variance_scaling_initializer())
        B = tf.get_variable("fc_{}_B".format(name), [out_size], initializer=tf.constant_initializer(bias_val))
        res = tf.matmul(x, W) + B
        
        if not is_output:
            res = tf.nn.relu(res)
        return res
    
    '''
    x is an input tensor
    Declare a name scope using the "name" parameter
    Within that scope:
      Create a W filter variable with the proper size
      Create a B bias variable with the proper size
      Multiply x by W and add b
      If is_output is False,
        Call the tf.nn.relu function
      Return the final op
    '''
    pass

In [ ]:
with tf.Graph().as_default():


    input_data = tf.placeholder( tf.float32, [1,32,32,3] )
    y_real = tf.placeholder(tf.int64, [1])

    h0 = conv( input_data, name="h0" )     # h0 = [1,32,32,64]
    h1 = conv( h0, name="h1" )             # h1 = [1,16,16,64]
    h2 = conv( h1, name="h2", stride = 2 ) # h2 = [1,4,4,64] 
    reshaping = h2.shape[0]*h2.shape[1]*h2.shape[2]*h2.shape[3] #get reshape val (1024)
    h2 = tf.reshape(h2,[1, int(reshaping)])           # h2 = [1024,1]
    h3 = fc( h2, name="h3" )               # h3 = [50,1]
    h4 = fc( h3, out_size = 10, name="h4", is_output = True) # h4 = [10,1]
    logit = h4

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y_real , logits = logit, name="cross_entropy")
    train_step = tf.train.AdamOptimizer(learning_rate=c).minimize(cross_entropy)

    prediction = tf.argmax(logit, axis=1) #y_real - tf.argmax()

    init = tf.global_variables_initializer()

    sess = tf.Session()
    writer = tf.summary.FileWriter("./tf_lab5", sess.graph)
    sess.run( init )

    epochs = 15
    tr_accuracy = []
    xe_tr = []
    te_accuracy = []
    xe_te = []

    for ep in tqdm(xrange(epochs)): 
        tr_intermed =  []
        te_intermed = []
        for im in xrange(len(train_labels)):
            _, xe, pred = sess.run([train_step, cross_entropy, prediction], feed_dict = {input_data: train_features[im], y_real: train_labels[im]})
            xe_tr.append(xe)
            if (pred[0]== 0):
                tr_intermed.append(1)
        tr_accuracy.append(float(len(tr_intermed))/len(train_labels))

        for im in xrange(len(test_labels)):
            xe, pred = sess.run([cross_entropy, prediction], feed_dict = {input_data: test_features[im], y_real: test_labels[im]})
            xe_te.append(xe)
            if (pred[0]== 0):
                te_intermed.append(1)
        te_accuracy.append(float(len(te_intermed))/len(test_labels))

    sess.close()
writer.close()



In [ ]:
import pylab as plt

plt.plot( tr_accuracy, label='Training')
plt.plot( te_accuracy, label='Test')
plt.title("Accuracy of Test vs Training")
plt.xlabel("Number of Epochs")
plt.ylabel("Percent Accuracy")
plt.legend(loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as pltt

tboard = pltt.imread('./tboardlab5.png')
pltt.figure(figsize=[15,15])
pltt.axis('off')
pltt.imshow(tboard)
pltt.show()

In [ ]:
""""
def progressive_predict(final_layer, labels_ph, name="progressive_predict"):
    with tf.variable_scope(name):
        logits = final_layer

        predictions = tf.cast(tf.argmax(logits, axis=1), tf.int32, name="predictions")
        accuracy, pred_update_op = tf.metrics.accuracy(
            labels=labels_ph,
            predictions= predictions,
            name="accuracy")

        return accuracy, pred_update_op


def measure_test(cross_entropy, accuracy, name="metrics_summary"):
    with tf.variable_scope(name):
        # create a summary for our cost and accuracy
        tf.summary.scalar("cross_entropy", cross_entropy)
        tf.summary.scalar("accuracy", accuracy)

        # merge all summaries into a single "operation" which we can execute in a session 
        summary_op = tf.summary.merge_all()
        return summary_op
    
def measure_train(cross_entropy, accuracy, name="metrics_summary"):
    with tf.variable_scope(name):
        # create a summary for our cost and accuracy
        tf.summary.scalar("cross_entropy", cross_entropy)
        tf.summary.scalar("accuracy", accuracy)

        # merge all summaries into a single "operation" which we can execute in a session 
        summary_op = tf.summary.merge_all()
        return summary_op

with tf.Graph().as_default():
    input_data = tf.placeholder( tf.float32, [1,32,32,3] )
    y_real = tf.placeholder(tf.int64, [1])

    h0 = conv( input_data, name="h0" )     # h0 = [1,32,32,64]
    h1 = conv( h0, name="h1" )             # h1 = [1,16,16,64]
    h2 = conv( h1, name="h2", stride = 2 ) # h2 = [1,4,4,64] 
    reshaping = h2.shape[0]*h2.shape[1]*h2.shape[2]*h2.shape[3] #get reshape val (1024)
    h2 = tf.reshape(h2,[1, int(reshaping)])           # h2 = [1024,1]
    h3 = fc( h2, name="h3" )               # h3 = [50,1]
    h4 = fc( h3, out_size = 10, name="h4", is_output = True) # h4 = [10,1]
    logit = h4
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y_real , logits = logit, name="cross_entropy")
    train = tf.train.AdamOptimizer(learning_rate=c).minimize(cross_entropy)

    accuracy, pred_update_op = progressive_predict(logit, y_real)
    get_test_metrics = measure_test(cross_entropy, accuracy, name="testing_set_metrics")
    get_train_metrics = measure_train(cross_entropy, accuracy, name="training_set_metrics")


    with tf.Session() as sess:
        j = 0
        Epoch = 15
        writer = tf.summary.FileWriter("output", sess.graph)

        for epoch in tqdm(range(Epoch)):
            for i in tqdm(range(len(labels))):
                _, _, train_summary = sess.run([train, pred_update_op, get_train_metrics], feed_dict = {input_data: resized_features[i,:], y_real: labels[i,:]})

                writer.add_summary(train_summary, epoch * i)

                if i % 4 == 0:
                    _, test_summary = sess.run([pred_update_op, get_test_metrics], feed_dict = {input_data: resized_features[j,:], y_real: labels[j,:]})

                    writer.add_summary(test_summary, epoch * j)
                    j += 1

        writer.close()
"""
